In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

print("TensorFlow:", tf.__version__)
print("GPU:", tf.config.list_physical_devices("GPU"))

TensorFlow: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

In [4]:
backbone = ResNet50(
    weights = "imagenet",
    include_top = False,
    input_shape = (256, 256, 3)

)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
for layers in backbone.layers:
  layers.trainable = False

In [7]:
layer_names = [
    "conv3_block4_out",
    "conv4_block6_out"
]

outputs = [backbone.get_layer(name).output for name in layer_names]
feature_extractor = Model(inputs = backbone.input, outputs = outputs)

In [8]:
# Sanity check
dummy = tf.random.normal((1, 256, 256, 3))
f1, f2 = feature_extractor(dummy)

print("Feature 1 shape:", f1.shape)
print("Feature 2 shape:", f2.shape)

Feature 1 shape: (1, 32, 32, 512)
Feature 2 shape: (1, 16, 16, 1024)
